In [1]:
import json
import random
import os
from tqdm.auto import tqdm

In [2]:
input_files=os.listdir("amazon-massive-dataset-1.0/1.0/data")
input_files.sort()

In [3]:
def get_utt_annotation(datum):
    
    utt=datum['annot_utt']
    domain=datum['scenario']
    intent=datum['intent']
    utt=utt.replace("["," [ ")
    utt=utt.replace("]"," ] ")
    utt=utt.split()
    
    token_list=[]
    tag_list=[]
    slot="unk"
    flag_in_slot=False
    triple=["","",[]]
    triple_list=[]
    count_triple=0
    
    for i in range(0,len(utt)):
        
        #print(utt[i])
        
        if((utt[i]=="[" and utt[i-1]!="]") or utt[i]==":" or utt[i]=="]"):
            
            continue
        
        if(utt[i-1]=="["):
            slot=utt[i]
            continue
            
        if(utt[i-1]==":"):
            token_list.append(utt[i])
            tag_list.append("B-"+domain+"-"+intent+"+"+slot)
            flag_in_slot=True
            triple[0]=domain+"-"+intent
            triple[1]=slot
            triple[2].append(utt[i])
            count_triple+=1
        elif(utt[i-1]=="]"):
            if(utt[i]!="["):
                token_list.append(utt[i])
                tag_list.append("O")
            flag_in_slot=False
            triple[2]=" ".join(triple[2])
            #print(triple)
            triple_list.append(triple)
            triple=["","",[]]
            
        elif(flag_in_slot):
            token_list.append(utt[i])
            tag_list.append("I-"+domain+"-"+intent+"+"+slot)
            flag_in_slot=True
            triple[2].append(utt[i])
        else:
            token_list.append(utt[i])
            tag_list.append("O")
            
            
    intent_list=[domain+"-"+intent]
    
    assert len(token_list)==len(tag_list)
    assert len(triple_list)==count_triple
    
    return [token_list,tag_list,intent_list,triple_list,[]]

def process_corpus(data):
    
    processed_data=[]
    all_intent_list=[]
    all_tag_list=[]

    for d in data:
        new_d=get_utt_annotation(d)
        processed_data.append(new_d)
        all_intent_list+=new_d[2]
        all_tag_list+=new_d[1]
        
    intent_vocab=list(set(all_intent_list))
    tag_vocab=list(set(all_tag_list))
    slot_vocab=["-".join(x.split('-')[1:]) for x in tag_vocab if x!="O"]
    slot_vocab=list(set(slot_vocab))

    random.shuffle(processed_data)
    
    train_len=int(len(processed_data)*0.8)
    valid_len=(len(processed_data)-train_len)//2
    test_len=len(processed_data)-train_len-valid_len

    train_data=processed_data[:train_len]
    valid_data=processed_data[train_len:train_len+valid_len]
    test_data=processed_data[train_len+valid_len:]
    
    
    return train_data,valid_data,test_data,intent_vocab,slot_vocab,tag_vocab

In [4]:
for infile in tqdm(input_files[:6]):
    
    data=[]

    with open("amazon-massive-dataset-1.0/1.0/data/"+infile,'r') as file:
        for line in file:
            data.append(eval(line.strip()))
            
    train_data,valid_data,test_data,intent_vocab,slot_vocab,tag_vocab=process_corpus(data)
    
    outdir="corpus/"+infile.split(".")[0]+"/data/processed_data/"
    
    if(not os.path.exists(outdir)):
        os.makedirs(outdir)
        
    with open(outdir+"train_data.json",'w') as file:
        json.dump(train_data,file,indent=4,ensure_ascii=False)

    with open(outdir+"val_data.json",'w') as file:
        json.dump(train_data,file,indent=4,ensure_ascii=False)

    with open(outdir+"test_data.json",'w') as file:
        json.dump(train_data,file,indent=4,ensure_ascii=False)

    with open(outdir+"intent_vocab.json",'w') as file:
        json.dump(intent_vocab,file,indent=4,ensure_ascii=False)

    with open(outdir+"slot_vocab.json",'w') as file:
        json.dump(slot_vocab,file,indent=4,ensure_ascii=False)

    with open(outdir+"tag_vocab.json",'w') as file:
        json.dump(tag_vocab,file,indent=4,ensure_ascii=False)